# Demo Tugas Akhir 2
---
Berikut adalah demo implementasi solusi Tugas Akhir 2 Pengujian Adversarial Robustness pada Model Bahasa Pra-latih dalam Domain Klasifikasi Teks. Dalam demo ini seluruh komponen dalam rancangan solusi telah diimplementasikan pada package yang diimport pada cell pertama.

In [1]:
import os, sys
import gc

gc.collect()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

from sklearn.metrics import accuracy_score
import swifter
import torch
device = torch.device("cuda")

from tqdm.notebook import tqdm
tqdm.pandas()

from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.get_args import get_args

from attack.adv_attack import attack
import os, sys

### Deklarasi variabel untuk pengujian
---
Pada pengujian kali ini dilakukan terhadap model IndoBERT-Base untuk task analisis sentimen dengan code-mixing Bahasa Perancis.

In [16]:
model_target = "IndoBERT"
downstream_task = "sentiment"
attack_strategy = "adversarial"
finetune_epoch = 10
num_sample = 10
exp_name = "demo"
perturbation_technique = "codemixing"
perturb_ratio = 0.6
dataset = "valid"
perturb_lang="fr"
seed=26092020

### Inisialisasi Pengujian
---
Pada tahap ini diinisialisasi random seed yang digunakan, kemudian inisialisasi model Universal Sentence Encoder, inisialisasi model, dan inisialisasi dataset.

In [17]:
set_seed(seed)
use = USE()

tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Tahap Pengujian
--- 
Dilakukan pengujian dengan parameter yang telah didefinisikan sebelumnya. Pengujian dilakukan secara iteratif pada dataset validasi menggunakan fungsi adversarial attack yang telah diimport pada package diatas.

In [25]:
if dataset == "valid":
    exp_dataset = valid_dataset.load_dataset(valid_path).sample(num_sample)
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
    
text,label = None,None

if downstream_task == 'sentiment':
    text = 'text'
    label = 'sentiment'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.text,
            true_label = row.sentiment,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.tweet,
            true_label = row.label,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )

100%|███████████████████████████████████████████████████████████████| 10/10 [00:47<00:00,  4.72s/it]


### Tahap Evaluasi
---
Pada tahap ini dilakukan evaluasi dari adversarial examples dan label prediksi yang telah dihasilkan pada cell sebelumnya. Pengujian dilakukan dengan melihat delta akurasi dan skor kemiripan semantik.

In [26]:
before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'delta_acc'] = before_attack-after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_semantic_sim'] = exp_dataset["perturbed_semantic_sim"].mean()
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()
# exp_dataset.to_csv(os.getcwd() + r'/result/seed'+str(seed)+"/"+str(dataset)+"/"+str(exp_name)+".csv", index=False)

In [27]:
exp_dataset

,text,sentiment,perturbed_text,perturbed_semantic_sim,pred_label,pred_proba,perturbed_label,perturbed_proba,translated_word(s),running_time(s),before_attack_acc,after_attack_acc,delta_acc,avg_semantic_sim,avg_running_time(s)
131,saya jadi tidak simpati lagi sama nikita mirza...,2,saya jadi tidak sympathie lagi sama nikita mir...,0.863004,2,"[0.0011, 0.0012, 0.9976]",2,"[0.0412, 0.0089, 0.9499]","{'blokir': 'bloc', 'simpati': 'sympathie', 'me...",1.4578,1.0,0.9,0.1,0.835196,4.71789
914,cocok buat makan buat yang punya teman banyak ...,0,cocok buat makan buat yang punya ami banyak ic...,0.901243,0,"[0.9982, 0.001, 0.0008]",0,"[0.9977, 0.0015, 0.0008]","{'enak': 'joli', 'nih': 'ici', 'soekarno': 'So...",2.5572,NaN,NaN,NaN,NaN,NaN
883,saya suka ur pizza dan calzone . karakter si r...,0,saya aimer tu pizza dan Calzone karakter si pa...,0.815185,0,"[0.9986, 0.0007, 0.0006]",0,"[0.9968, 0.0017, 0.0016]","{'banget': 'très', 'calzone': 'Calzone', 'suka...",4.8165,NaN,NaN,NaN,NaN,NaN
690,iya pak untuj transfer data ke komputer,1,oui pak pour transférer data ke komputer,0.759898,1,"[0.0016, 0.9972, 0.0012]",1,"[0.0016, 0.9751, 0.0233]","{'untuj': 'pour', 'transfer': 'transférer', 'i...",1.9375,NaN,NaN,NaN,NaN,NaN
83,ini adalaha salah satu tempat makan yang wajib...,0,ini est un salah satu tempat manger yang doit ...,0.891654,0,"[0.9983, 0.001, 0.0007]",0,"[0.9972, 0.0015, 0.0013]","{'enak': 'joli', 'adalaha': 'est un', 'restour...",6.7562,NaN,NaN,NaN,NaN,NaN
408,"saya memesan iga bakar , setelah datang ternya...",2,saya memesan iga brûler setelah datang ternyat...,0.801182,2,"[0.003, 0.0011, 0.9958]",1,"[0.0582, 0.5116, 0.4302]","{'mengecewakan': 'décevant', 'sungguh': 'série...",3.2675,NaN,NaN,NaN,NaN,NaN
1132,kami mengunjungi tempat ini karena lokasi nya ...,0,kami visiter tempat ini karena lokasi le sien ...,0.920251,0,"[0.9983, 0.0011, 0.0007]",0,"[0.9959, 0.0023, 0.0018]","{'mahal': 'chere', 'fi': 'Fi', 'wi': 'Wi', 'ba...",6.5424,NaN,NaN,NaN,NaN,NaN
654,di sini saya suka lontong laksa dan lontong op...,0,di sini saya aimer gâteau de riz laksa dan gât...,0.772177,0,"[0.9985, 0.001, 0.0005]",0,"[0.9954, 0.0034, 0.0012]","{'suka': 'aimer', 'kroket': 'croquette', 'top'...",7.6449,NaN,NaN,NaN,NaN,NaN
565,the harvest ini setahu saya sudah ada beberapa...,0,la récolte ini Pour autant que saya sudah ada ...,0.802164,0,"[0.9969, 0.0009, 0.0021]",0,"[0.9885, 0.0042, 0.0074]","{'dicoba': 'essayer', 'harvest': 'récolte', 't...",7.0313,NaN,NaN,NaN,NaN,NaN
333,makan malam kali ini masih di sekitar di jalan...,0,manger malam temps ini masih di sekitar di jal...,0.825202,0,"[0.9984, 0.0007, 0.0008]",0,"[0.9965, 0.0014, 0.0021]","{'garing': 'croustillant', 'setiabudhi': 'Seti...",5.1676,NaN,NaN,NaN,NaN,NaN
